In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext("local", "ReadFromHDFS")

22/05/12 20:37:27 WARN Utils: Your hostname, Hocines-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.164 instead (on interface en0)
22/05/12 20:37:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/12 20:37:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/12 20:37:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/12 20:37:28 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/05/12 20:37:28 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
# 1 - Load movies data into spark rdd tuple with (movieid, title, genres) format

rddMovies = sc.textFile("hdfs://localhost:9000/ml-latest-small/movies.csv")

# get first line
rddMovies.first()

22/05/12 20:37:35 WARN BlockReaderFactory: I/O error constructing remote block reader.
java.nio.channels.ClosedByInterruptException
	at java.nio.channels.spi.AbstractInterruptibleChannel.end(AbstractInterruptibleChannel.java:202)
	at sun.nio.ch.SocketChannelImpl.connect(SocketChannelImpl.java:658)
	at org.apache.hadoop.net.SocketIOWithTimeout.connect(SocketIOWithTimeout.java:192)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:586)
	at org.apache.hadoop.hdfs.DFSClient.newConnectedPeer(DFSClient.java:3025)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.nextTcpPeer(BlockReaderFactory.java:826)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.getRemoteBlockReaderFromTcp(BlockReaderFactory.java:751)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.build(BlockReaderFactory.java:381)
	at org.apache.hadoop.hdfs.DFSInputStream.getBlockReader(DFSInputStream.java:755)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:685)
	at org.apac

'movieId,title,genres'

In [4]:
#2 - Count number of movies based on Genres fields, 
# use Adventure, Romance, Comedy as specific genres, get the count of movies on each genres

rddMovies.count()


#wordCountRdd = sc.textFile("hdfs://localhost:9000/books/book-war-and-peace.txt")\
                # .map (lambda line: to_ascii(line))\
               #   .map (lambda line: line.strip().lower())\
                # .map (lambda line: line.split(" "))\
               #  .flatMap(lambda elements: elements)\
               #  .filter (lambda word: word != "")\
               #  .map (lambda word: (word, 1))\
                # .reduceByKey(lambda acc, value: acc + value)

#wordCountRdd.take(10)

9743

In [5]:
# 3 - Load ratings.csv into tuple (userid, movieid, rating, timestamp) format

rddRatings = sc.textFile("hdfs://localhost:9000/ml-latest-small/ratings.csv")

# get first line
rddRatings.first()


22/05/12 20:38:54 WARN BlockReaderFactory: I/O error constructing remote block reader.
java.nio.channels.ClosedByInterruptException
	at java.nio.channels.spi.AbstractInterruptibleChannel.end(AbstractInterruptibleChannel.java:202)
	at sun.nio.ch.SocketChannelImpl.connect(SocketChannelImpl.java:658)
	at org.apache.hadoop.net.SocketIOWithTimeout.connect(SocketIOWithTimeout.java:192)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:586)
	at org.apache.hadoop.hdfs.DFSClient.newConnectedPeer(DFSClient.java:3025)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.nextTcpPeer(BlockReaderFactory.java:826)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.getRemoteBlockReaderFromTcp(BlockReaderFactory.java:751)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.build(BlockReaderFactory.java:381)
	at org.apache.hadoop.hdfs.DFSInputStream.getBlockReader(DFSInputStream.java:755)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:685)
	at org.apac

'userId,movieId,rating,timestamp'

In [2]:
# 4 - Remove the rating which is lower than 1.0 from the ratings.csv

# use moviesRdd2 and filter, print the movies id  betwen 100 to 105
filteredRdd = moviesRdd2.filter (lambda t: t[0] >= 100 and t[0] <= 105)
print(filteredRdd.take(6))


In [ ]:
# to skip first line, 
headerLine = rdd.first()
# no header data in rddContent
rddContent = rdd.filter (lambda line: line != headerLine)

print (rddContent.take(5) )

In [ ]:
# 5 - Use reduceByKey, count the ratings for the movies based on movieid, for example, if a movie received 10 ratings, 
#the output should be a tuple (movieId, 10)


# convert word into (key,value) rdd (spark, 1) for reduceByKey
pairRdd = wordRdd.map (lambda word: (word, 1))
pairRdd.take(5)

# get word count using reduceByKey
# transformation
wordCountRdd = pairRdd.reduceByKey(lambda acc, value: acc + value)
wordCountRdd.take(5)

In [ ]:
# write the result into text file in hdfs
# saveAsTextFile is an ACTION Method
# word-count-results is a folder, inside we will shall partition files

wordCountRdd.saveAsTextFile ("hdfs://localhost:9000/word-count-results")

In [3]:
### check flod file on github 

# 6 - Use Fold or FoldByKey, find the sum/or avg rating received by a movie in rating.csv


# fold with aggregate with start value 0
# fold is action method
# fold works with each partition first, calculate add function on each partition
# + it will apply result of all paritions into again another folder
# return value of add is passed as input with next number in seq

# after processing data from partition 0, it got result 150
# then it will apply add function across partition result  acc 0 value 150
result = rdd.fold (0, add) # output shall be 150 = 0 + 10 + 20 + 30 + 40 + 50
print("result", result)


In [4]:
# 7 - Convert timestamp in rating.csv into Python date time object



In [ ]:

# check s011 and s012

# 8 - Join ratings and movies based on movie id, so that we could see movie title and genres for the given movie


# joins ,

sectors = [
    # tuple, the key is used for join purpose
    # sector id, sector name
    (0, "AUTOMOTIVE"),
    (1, "TEXTILE"),
    (2, "IT"),
    (4, "MANUFACTURING")
]

stocks = [
    # key   is used for join purpose
    
    # key is 0, 1, 4 and value is  ("TSLA", 100), SYM1, SYM2
    # sector id, symbol, price
    (0, ("TSLA", 100) ),
    (0, ("GM", 40 ) ),
    (1, ("SYM1", 45) ),
    (4, ("SYM2", 67) )
]

sectorRdd = sc.parallelize(sectors)
stocksRdd = sc.parallelize(stocks)
# join based on rdd keys it has to match sectorRdd sector id with stocksRdd sector id
joinRdd = sectorRdd.join(stocksRdd)

joinRdd.collect()

In [5]:
# check from dataframe s52

# 9 - Use sort on question number 5, sort the movie rating count by descending order so that highest rated movie should be on the top



from pyspark.sql.functions import desc, asc
rddSort  = rdd5.sort(desc("price"))


# alternatively use dataframe columns if we have df reference
df = productDf.sort (productDf.price.asc())
df.show()
# desc
df = productDf.sort (productDf.price.desc())
df.show()


In [ ]:
# check s008 s009


# to skip first line, 
headerLine = rdd.first()
# no header data in rddContent
rddContent = rdd.filter (lambda line: line != headerLine)

print (rddContent.take(5) )